## Loading files into mongodb

In [1]:
from pymongo import MongoClient # Install for using this lib.
import subprocess

In [2]:
dbname = 'test'
collection_name1 = 'reviews'
collection_name2 = 'business'
collection_name3 = 'users'

path = "/Users/neerjadoshi/msan/Yelp/dataset/"

input_file_name1 = path+"review.json"
input_file_name2 = path+'business.json'
input_file_name3 = path+'user.json'



In [3]:
def import_query(dbname, collection_name, input_file_name):
    mongoimport_query =  'mongoimport --db ' + dbname + ' --collection ' + collection_name + ' --file ' + input_file_name
    #COMPLETE THIS.
    return mongoimport_query
 
#Create connection 
client = MongoClient() #default-localhost:27017
#Connect to database
db = client[dbname]

#Drop table.
db[collection_name1].drop()
db[collection_name2].drop()
db[collection_name3].drop()

#Insert all data from the input_file_name.

In [4]:
mongoimport_query = import_query(dbname, collection_name1, input_file_name1)
subprocess.call(mongoimport_query,shell=True)

mongoimport_query = import_query(dbname, collection_name2, input_file_name2)
subprocess.call(mongoimport_query,shell=True)

mongoimport_query = import_query(dbname, collection_name3, input_file_name3)
subprocess.call(mongoimport_query,shell=True)

0

In [24]:
# db.reviews.find()

## Keep only restaurants

In [52]:
restaurants = db.business.find({ '$or' : [ { "attributes.RestaurantsReservations" : {'$exists' : 'true'} }, { "attributes.RestaurantsDelivery" : {'$exists':'true'}}, 
                            { "attributes.RestaurantsAttire" : {'$exists':'true'}}, { "attributes.RestaurantsCounterService" : {'$exists':'true'}}, 
                            { "attributes.RestaurantsGoodForGroups" : {'$exists':'true'}}, { "attributes.RestaurantsPriceRange2" : {'$exists':'true'}}, 
                            { "attributes.RestaurantsTableService" : {'$exists':'true'}}, { "attributes.RestaurantsTakeOut" : {'$exists':'true'}}, 
                            { "attributes.GoodForMeal" : {'$exists':'true'}}]})

In [53]:
len(list(restaurants))

99566

#### Finding restuarants having review counts greater than 25

In [33]:
restaurants = db.business.find({ '$or' : [ { "attributes.RestaurantsReservations" : {'$exists' : 'true'} }, { "attributes.RestaurantsDelivery" : {'$exists':'true'}}, 
                            { "attributes.RestaurantsAttire" : {'$exists':'true'}}, { "attributes.RestaurantsCounterService" : {'$exists':'true'}}, 
                            { "attributes.RestaurantsGoodForGroups" : {'$exists':'true'}}, { "attributes.RestaurantsPriceRange2" : {'$exists':'true'}}, 
                            { "attributes.RestaurantsTableService" : {'$exists':'true'}}, { "attributes.RestaurantsTakeOut" : {'$exists':'true'}}, 
                            { "attributes.GoodForMeal" : {'$exists':'true'}}], 
                    "review_count":{"$gt": 25}})

In [34]:
rest = list(restaurants)

In [35]:
# Collecting business ID's
business_ids = []
for i in range(len(rest)):
    business_ids.append(rest[i]['business_id'])

In [9]:
len(business_ids)

30152

In [10]:
# list(db.business.find({ '$or' : [ { "attributes.RestaurantsReservations" : {'$exists' : 'true'} }, { "attributes.RestaurantsDelivery" : {'$exists':'true'}}, 
#                             { "attributes.RestaurantsAttire" : {'$exists':'true'}}, { "attributes.RestaurantsCounterService" : {'$exists':'true'}}, 
#                             { "attributes.RestaurantsGoodForGroups" : {'$exists':'true'}}, { "attributes.RestaurantsPriceRange2" : {'$exists':'true'}}, 
#                             { "attributes.RestaurantsTableService" : {'$exists':'true'}}, { "attributes.RestaurantsTakeOut" : {'$exists':'true'}}, 
#                             { "attributes.GoodForMeal" : {'$exists':'true'}}]}))

#### Filtering users (having reviews count greater than 25)

In [36]:
filtered_users = db.users.find({"review_count":{"$gt": 25}})

In [37]:
user = list(filtered_users )

In [38]:
user_ids = []
for i in range(len(user)):
    user_ids.append(user[i]['user_id'])

In [39]:
len(user_ids)

203189

#### Filtering reviews basis above user_id & business_id and stars

In [23]:
# len(list(db.reviews.find()))

In [ ]:
filtered_reviews = db.reviews.find({"user_id": {'$in':user_ids}, "business_id": {'$in':business_ids}, 
                                    "stars": {'$in':[1,2,3,4,5]}})

In [21]:
# review = list(filtered_reviews )

In [22]:
# len(review)

1727718

**Saving filtered reviews as a table:**

In [ ]:
db.filtered_reviews.drop()
result = db.filtered_reviews.insert_many(filtered_reviews)
# result.inserted_ids



## Load tables into pyspark

In [25]:
reviews = spark.read.format("com.mongodb.spark.sql.DefaultSource").option("uri","mongodb://127.0.0.1/test.filtered_reviews").load()

business = spark.read.format("com.mongodb.spark.sql.DefaultSource").option("uri","mongodb://127.0.0.1/test.business").load()

users = spark.read.format("com.mongodb.spark.sql.DefaultSource").option("uri","mongodb://127.0.0.1/test.users").load()

In [26]:
reviews.show(1)

+--------------------+--------------------+----+----------+-----+--------------------+-----+--------------------+------+--------------------+
|                 _id|         business_id|cool|      date|funny|           review_id|stars|                text|useful|             user_id|
+--------------------+--------------------+----+----------+-----+--------------------+-----+--------------------+------+--------------------+
|[5a5baf10cf6cfeac...|jQsNFOzDpxPmOurSW...|   0|2015-03-26|    0|i5UwUPlQFPLcE8p2g...|    4|For being fairly ...|     1|WZXp9-V2dqRRJqhGg...|
+--------------------+--------------------+----+----------+-----+--------------------+-----+--------------------+------+--------------------+
only showing top 1 row



In [27]:
# reviews.count()

1727718

In [28]:
reviews = reviews.select(['business_id', 'user_id', 'text', 'stars'])

In [29]:
business = business.select(['business_id', 'stars'])

In [30]:
users = users.select(['user_id'])